# Word2GM Training Data Pipeline

**Pipeline: Corpus file → TFRecord training artifacts (triplets and vocabulary)**

Use this notebook to prepare a Google 5gram corpora for Word2GM skip-gram training.

## Pipeline Workflow

1. **Input**: Preprocessed corpus file (e.g., `2019.txt`) in `/vast` NVMe storage
2. **Processing**: TensorFlow-native filtering, vocabulary building, and triplet generation
3. **Output**: TFRecord artifacts in organized subdirectories (e.g., `2019_artifacts/`)

### **Artifact Storage**
The pipeline creates year-specific subdirectories alongside the original text corpora:
<pre>
/vast/edk202/NLP_corpora/.../data/
├── 2018.txt
├── 2019.txt
├── 2020.txt
├── 2018_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
├── 2019_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
└── 2020_artifacts/
    ├── triplets.tfrecord.gz
    └── vocab.tfrecord.gz
</pre>

## Set Up for Data Preparation

In [1]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_data_preprocessing_notebook, enable_autoreload

# Enable autoreload for development
enable_autoreload()

# Set up environment (CPU-only for data preprocessing)
env = setup_data_preprocessing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
batch_prepare_training_data = env['batch_prepare_training_data']
print_resource_summary = env['print_resource_summary']

<pre>Autoreload enabled</pre>

<pre>Project root: /scratch/edk202/word2gm-fast
TensorFlow version: 2.19.0
Device mode: CPU-only</pre>

<pre>Data preprocessing environment ready!</pre>

## Print Resource Summary

In [2]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: cm048.hpc.nyu.edu

Job Allocation:
   CPUs: 48
   Memory: 125.0 GB
   Requested partitions: short
   Running on: SSH failed: Host key verification failed.
   Job ID: 63410489
   Node list: cm048

GPU Information:
   Error: NVML Shared Library Not Found

TensorFlow GPU Detection:
   TensorFlow detects 0 GPU(s)
   Built with CUDA: True
============================================================</pre>

## Prepare Corpora

Here, we run the data-preparation pipeline from start to finish — reading preprocessed ngram corpora, generating all valid triplets, extracting the vocabulary, and saving the triplets and vocabulary as `tfrecord` files.

### Options for Data Preparation

You can control which years are processed and how the batch preparation runs by adjusting the arguments to `batch_prepare_training_data`:

**Ways to specify years:**
- `year_range="2010"` — Process a single year (e.g., only 2010).
- `year_range="2010,2012,2015"` — Process a comma-separated list of years.
- `year_range="2010-2015"` — Process a range of years, inclusive (2010 through 2015).
- `year_range="2010,2012-2014,2016"` — Combine individual years and ranges (2010, 2012, 2013, 2014, 2016).

**Other options:**
- `compress` — If `True`, output TFRecords are gzip-compressed. If `False`, output is uncompressed.
- `show_progress` — If `True`, display a progress bar for each year.
- `show_summary` — If `True`, print a summary of the processed data for each year.
- `use_multiprocessing` — If `True`, process years in parallel using multiple CPU cores (recommended for large datasets).

See the function docstring or source for more advanced options.

In [ ]:
# Configuration
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"

downsample_threshold = 1e-5  # Typical values: 1e-3 (conservative), 1e-4 (moderate), 1e-5 (aggressive)

# Process year(s)
results = batch_prepare_training_data(
    corpus_dir=corpus_dir,
    year_range="1801-1900",
    compress=False,
    show_progress=True,
    show_summary=True,
    use_multiprocessing=True,
    downsample_threshold=downsample_threshold
)

Requested 100 years, found 100 corpus files.

PARALLEL BATCH PROCESSING
Processing 100 years
Using 48 parallel workers
Estimated speedup: 48.0x


1804 complete (1/100): 32,051 triplets, 9,769 vocab, 79.7s
1802 complete (2/100): 21,569 triplets, 8,487 vocab, 137.9s
1808 complete (3/100): 83,265 triplets, 13,193 vocab, 222.3s
1805 complete (4/100): 49,730 triplets, 11,609 vocab, 242.8s
1801 complete (5/100): 41,106 triplets, 11,000 vocab, 244.9s
1814 complete (6/100): 84,587 triplets, 14,965 vocab, 261.1s
1803 complete (7/100): 96,772 triplets, 14,473 vocab, 349.8s
1817 complete (8/100): 155,413 triplets, 17,803 vocab, 367.2s
1807 complete (9/100): 100,828 triplets, 14,626 vocab, 392.3s
1816 complete (10/100): 112,225 triplets, 16,144 vocab, 399.8s
1806 complete (11/100): 106,612 triplets, 14,124 vocab, 456.8s
1813 complete (12/100): 143,347 triplets, 17,428 vocab, 485.4s
1812 complete (13/100): 199,177 triplets, 18,423 vocab, 526.2s
1809 complete (14/100): 212,783 triplets, 19,226 vocab, 537.2s
1815 complete (15/100): 184,543 triplets, 17,905 vocab, 571.2s
1811 complete (16/100): 196,589 triplets, 17,916 vocab, 695.1s
1827 comple